In [2]:
from gurobipy import Model, GRB

M = 3654.0
T_max = 180.0

cal_per_hour = {
    'walk': 342.0,
    'cycle': 903.6,
    'swim': 542.2,
    'run': 564.8,
    'strength': 300.0
}

m = Model("fitness_plan")


H = {}
for act in cal_per_hour:
    H[act] = m.addVar(lb=0.0, name=f"H_{act}")

N1 = m.addVar(lb=411.0, ub=594.0, name="carbs")      # carbs
N2 = m.addVar(lb=73.0, name="protein")               # protein
N3 = m.addVar(lb=81.0, ub=142.0, name="fat")         # fat
N4 = m.addVar(lb=51.0, name="fiber")                 # fiber
N5 = m.addVar(lb=0.0, name="other_cals")             # other calories

m.update()

CIN_d = 4*N1 + 4*N2 + 9*N3 + 2*N4 + N5

COUT = (
    cal_per_hour['walk']    * H['walk'] +
    cal_per_hour['cycle']   * H['cycle'] +
    cal_per_hour['swim']    * H['swim'] +
    cal_per_hour['run']     * H['run'] +
    cal_per_hour['strength']* H['strength']
)

m.setObjective(COUT, GRB.MAXIMIZE)

m.addConstr(sum(H[a] for a in H) <= T_max, name="time_limit")
m.addConstr(H['strength'] >= 6.0, name="min_strength_hours")
m.addConstr(
    H['walk'] + H['cycle'] + H['swim'] + H['run'] >= 7.5,
    name="min_aerobic_hours"
)

m.addConstr(CIN_d - M - COUT/30.0 >= -750.0, name="deficit_low")
m.addConstr(CIN_d - M - COUT/30.0 <= -500.0, name="deficit_high")

m.Params.OutputFlag = 0  
m.optimize()

if m.status == GRB.OPTIMAL:
    print(f"Optimal monthly calories burned (COUT): {COUT.getValue():.2f}")
    for a in H:
        print(f"{a}: {H[a].X:.2f} hours/month")

    print(f"Daily calories intake CIN_d: {CIN_d.getValue():.2f}")
    print(f"carbs  (g/day): {N1.X:.2f}")
    print(f"protein(g/day): {N2.X:.2f}")
    print(f"fat    (g/day): {N3.X:.2f}")
    print(f"fiber  (g/day): {N4.X:.2f}")
    print(f"other cals/day: {N5.X:.2f}")

    D = CIN_d.getValue() - M - COUT.getValue()/30.0
    print(f"Daily deficit D: {D:.2f} kcal/day")
else:
    print("Model not optimal, status =", m.status)


Optimal monthly calories burned (COUT): 159026.40
walk: 0.00 hours/month
cycle: 174.00 hours/month
swim: 0.00 hours/month
run: 0.00 hours/month
strength: 6.00 hours/month
Daily calories intake CIN_d: 8454.88
carbs  (g/day): 411.00
protein(g/day): 1494.97
fat    (g/day): 81.00
fiber  (g/day): 51.00
other cals/day: 0.00
Daily deficit D: -500.00 kcal/day
